In [5]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import sympy as sm
from sympy.physics.vector import dynamicsymbols
from sympy import diff, Symbol, Derivative

# step 1: find the Lagrangian

# ds is sling linear density, z is position along the sling
La, Ls, mp, mb, Ia, rc, g, t, ds, z = sm.symbols('La Ls mp mb Ia rc g t ds z')
theta, psi, thetad, psid = dynamicsymbols('theta psi thetad psid')
print(theta)

xp = -La*sm.sin(theta) - Ls*sm.sin(psi - theta)
yp = La*sm.cos(theta) - Ls*sm.cos(psi - theta)

xz = -La*sm.sin(theta) - z*sm.sin(psi - theta)
yz = La*sm.cos(theta) - z*sm.cos(psi - theta)
vxz = sm.diff(xz, t)
vyz = sm.diff(yz, t)


axz = sm.diff(vxz, t)
ayz = sm.diff(vyz, t)
sx_indef = sm.integrate(axz, z)
sy_indef = sm.integrate(ayz, z)
sx = sm.simplify(sx_indef.subs(z, Ls) - sx_indef.subs(z, 0))
sy = sm.simplify(sy_indef.subs(z, Ls) - sy_indef.subs(z, 0))
# used for calculating force on the tip of the arm and maximum load on the sling
# sx and sy are each multiplied by the linear density of the sling to yield the x and y contributions of
# sling mass to tip load. Usually these contributions will be small in comparison to the mp*ap term

v2z = sm.simplify(vxz**2 + vyz**2) # square of the speed of a point distance z along the sling from the tip
Ts_indef = 0.5*ds*sm.integrate(v2z, z)
Ts = sm.simplify(Ts_indef.subs(z, Ls) - Ts_indef.subs(z, 0)) # kinetic energy of the sling

T = 0.5*Ia*(sm.diff(theta, t))**2 + 0.5*mp*((sm.diff(xp, t))**2 + (sm.diff(yp, t))**2) + Ts

ybcom = rc*sm.cos(theta)
Vs = ds*Ls*g*(La*sm.cos(theta) - 0.5*Ls*sm.cos(psi - theta))
V = mp*g*yp + mb*g*ybcom + Vs

L = T - V
print("vxz = ", vxz)
print("vyz = ", vyz)
print("axz = ", axz)
print("ayz = ", ayz)
print("sx = ", sx)
print("sy = ", sy)
print("v2z = ", v2z)
print("Ts = ", Ts)
print("Vs = ", Vs)
print("vpx = ", sm.diff(xp, t))
print("vpy = ", sm.diff(yp, t))

V

theta(t)
vxz =  -La*cos(theta(t))*Derivative(theta(t), t) - z*(Derivative(psi(t), t) - Derivative(theta(t), t))*cos(psi(t) - theta(t))
vyz =  -La*sin(theta(t))*Derivative(theta(t), t) + z*(Derivative(psi(t), t) - Derivative(theta(t), t))*sin(psi(t) - theta(t))
axz =  La*sin(theta(t))*Derivative(theta(t), t)**2 - La*cos(theta(t))*Derivative(theta(t), (t, 2)) + z*(Derivative(psi(t), t) - Derivative(theta(t), t))**2*sin(psi(t) - theta(t)) - z*(Derivative(psi(t), (t, 2)) - Derivative(theta(t), (t, 2)))*cos(psi(t) - theta(t))
ayz =  -La*sin(theta(t))*Derivative(theta(t), (t, 2)) - La*cos(theta(t))*Derivative(theta(t), t)**2 + z*(Derivative(psi(t), t) - Derivative(theta(t), t))**2*cos(psi(t) - theta(t)) + z*(Derivative(psi(t), (t, 2)) - Derivative(theta(t), (t, 2)))*sin(psi(t) - theta(t))
sx =  Ls*(2*La*(sin(theta(t))*Derivative(theta(t), t)**2 - cos(theta(t))*Derivative(theta(t), (t, 2))) + Ls*(sin(psi(t) - theta(t))*Derivative(psi(t), t)**2 - 2*sin(psi(t) - theta(t))*Derivative(psi(t), t)*

Ls*ds*g*(La*cos(theta(t)) - 0.5*Ls*cos(psi(t) - theta(t))) + g*mb*rc*cos(theta(t)) + g*mp*(La*cos(theta(t)) - Ls*cos(psi(t) - theta(t)))

In [3]:
# step 2: apply the Euler-Lagrange equations, and find the differential equations describing the motion
L = L.subs(Derivative(theta), thetad)
L = L.subs(Derivative(psi), psid)
print("L = ", L, "\n")
T_drive, thetadd, psidd = sm.symbols('T_drive thetadd psidd') # drive torque, applied about main pivot

eq_EL0 = sm.diff(sm.diff(L, thetad), t) - sm.diff(L, theta) + T_drive # equation for theta
# eq_EL0 and eq_EL1 are equal to zero. Plus sign on T_drive is because applying positive torque will cause the arm to rotate
# backwards (CCW, using the definition of theta shown on 2017-11-06 pg2). So, T_drive should always be positive, using
# the above equation
eq_EL1 = sm.diff(sm.diff(L, psid), t) - sm.diff(L, psi) # no generalized force term, this angle is unactuated

#not using LagrangesMethod because it's almost completely undocumented
# now have two 2nd order ODEs represented as two eqns, both equal to zero
# just need to rearrange to isolate thetadd in one and psidd in the other
# the other 2 eqns come from the definitions of the derivatives, as per the usual dance for 2nd order systems


eq00 = eq_EL0
eq10 = eq_EL1

eq_EL0 = eq_EL0.subs([(Derivative(thetad, t), thetadd), (Derivative(psid, t), psidd), (Derivative(theta, t), thetad), 
                      (Derivative(psi, t), psid)])
eq_EL1 = eq_EL1.subs([(Derivative(thetad, t), thetadd), (Derivative(psid, t), psidd), (Derivative(theta, t), thetad), 
                      (Derivative(psi, t), psid)])

print("eq_EL0 = ", eq_EL0, "\n")
print("eq_EL1 = ", eq_EL1, "\n")

psidd_sol0 = list(sm.solveset(eq_EL1, psidd))[0]
eq_EL0_nopsidd = eq_EL0.subs(psidd, psidd_sol0)
thetadd_iso = list(sm.solveset(eq_EL0_nopsidd, thetadd))[0]
eq_EL1_nothetadd = eq_EL1.subs(thetadd, thetadd_iso)
psidd_iso = list(sm.solveset(eq_EL1_nothetadd, psidd))[0]
eq10

L =  0.5*Ia*thetad(t)**2 - Ls*ds*g*(La*cos(theta(t)) - 0.5*Ls*cos(psi(t) - theta(t))) + 0.166666666666667*Ls*ds*(3*La**2*thetad(t)**2 + 3*La*Ls*(psid(t) - thetad(t))*thetad(t)*cos(psi(t)) + Ls**2*(psid(t)**2 - 2*psid(t)*thetad(t) + thetad(t)**2)) - g*mb*rc*cos(theta(t)) - g*mp*(La*cos(theta(t)) - Ls*cos(psi(t) - theta(t))) + 0.5*mp*((-La*thetad(t)*sin(theta(t)) + Ls*(psid(t) - thetad(t))*sin(psi(t) - theta(t)))**2 + (-La*thetad(t)*cos(theta(t)) - Ls*(psid(t) - thetad(t))*cos(psi(t) - theta(t)))**2) 

eq_EL0 =  1.0*Ia*thetadd + Ls*ds*g*(-La*sin(theta(t)) - 0.5*Ls*sin(psi(t) - theta(t))) + 0.166666666666667*Ls*ds*(6*La**2*thetadd - 3*La*Ls*thetadd*cos(psi(t)) + 3*La*Ls*(psidd - thetadd)*cos(psi(t)) - 3*La*Ls*(psid(t) - thetad(t))*psid(t)*sin(psi(t)) + 3*La*Ls*psid(t)*thetad(t)*sin(psi(t)) + Ls**2*(-2*psidd + 2*thetadd)) + T_drive - g*mb*rc*sin(theta(t)) + g*mp*(-La*sin(theta(t)) - Ls*sin(psi(t) - theta(t))) - 0.5*mp*((-La*thetad(t)*sin(theta(t)) + Ls*(psid(t) - thetad(t))*sin(psi(t) - th

0.5*La*Ls**2*ds*(psid(t) - thetad(t))*thetad(t)*sin(psi(t)) + 0.5*Ls**2*ds*g*sin(psi(t) - theta(t)) + 0.166666666666667*Ls*ds*(-3*La*Ls*thetad(t)*sin(psi(t))*Derivative(psi(t), t) + 3*La*Ls*cos(psi(t))*Derivative(thetad(t), t) + Ls**2*(2*Derivative(psid(t), t) - 2*Derivative(thetad(t), t))) + Ls*g*mp*sin(psi(t) - theta(t)) - 0.5*mp*(2*Ls*(-La*thetad(t)*sin(theta(t)) + Ls*(psid(t) - thetad(t))*sin(psi(t) - theta(t)))*(psid(t) - thetad(t))*cos(psi(t) - theta(t)) + 2*Ls*(-La*thetad(t)*cos(theta(t)) - Ls*(psid(t) - thetad(t))*cos(psi(t) - theta(t)))*(psid(t) - thetad(t))*sin(psi(t) - theta(t))) + 0.5*mp*(2*Ls*(-La*thetad(t)*sin(theta(t)) + Ls*(psid(t) - thetad(t))*sin(psi(t) - theta(t)))*(Derivative(psi(t), t) - Derivative(theta(t), t))*cos(psi(t) - theta(t)) + 2*Ls*(-La*thetad(t)*cos(theta(t)) - Ls*(psid(t) - thetad(t))*cos(psi(t) - theta(t)))*(Derivative(psi(t), t) - Derivative(theta(t), t))*sin(psi(t) - theta(t)) - 2*Ls*(La*thetad(t)*sin(theta(t))*Derivative(theta(t), t) - La*cos(theta(

In [11]:
print("psidd_iso = ", psidd_iso)
print("\nthetadd_iso = ", thetadd_iso)

psidd_iso =  (0.166666666666667*Ia*La*Ls**2*ds**2*thetad(t)**2*sin(psi(t)) + 0.833333333333333*Ia*La*Ls*ds*mp*thetad(t)**2*sin(psi(t)) + 1.0*Ia*La*mp**2*thetad(t)**2*sin(psi(t)) - 0.166666666666667*Ia*Ls**2*ds**2*g*sin(psi(t) - theta(t)) - 0.833333333333333*Ia*Ls*ds*g*mp*sin(psi(t) - theta(t)) - 1.0*Ia*g*mp**2*sin(psi(t) - theta(t)) + 0.166666666666667*La**3*Ls**3*ds**3*thetad(t)**2*sin(psi(t)) + 1.0*La**3*Ls**2*ds**2*mp*thetad(t)**2*sin(psi(t)) + 1.83333333333333*La**3*Ls*ds*mp**2*thetad(t)**2*sin(psi(t)) + 1.0*La**3*mp**3*thetad(t)**2*sin(psi(t)) - 0.0833333333333333*La**2*Ls**4*ds**3*psid(t)**2*sin(psi(t))*cos(psi(t)) + 0.166666666666667*La**2*Ls**4*ds**3*psid(t)*thetad(t)*sin(psi(t))*cos(psi(t)) - 0.166666666666667*La**2*Ls**4*ds**3*thetad(t)**2*sin(psi(t))*cos(psi(t)) - 0.166666666666667*La**2*Ls**3*ds**3*g*sin(psi(t))*cos(theta(t)) - 0.583333333333333*La**2*Ls**3*ds**2*mp*psid(t)**2*sin(psi(t))*cos(psi(t)) + 1.16666666666667*La**2*Ls**3*ds**2*mp*psid(t)*thetad(t)*sin(psi(t))*cos(

In [18]:
r2z = xz**2 + yz**2
Is_indef = ds*sm.integrate(r2z, z)
Is = sm.simplify(Is_indef.subs(z, Ls) - Is_indef.subs(z, 0))
Ip = sm.simplify(mp*r2z.subs(z, Ls))
It = Ia + Ip + Is # total inertia of the arm-sling-projectile system for platform option
print("It = ", It)

It =  Ia + Ls*ds*(3*La**2 - 3*La*Ls*cos(psi(t)) + Ls**2)/3 + mp*(La**2 - 2*La*Ls*cos(psi(t)) + Ls**2)


In [28]:
print(vxz.subs([(z, Ls)]))
print(vyz.subs([(z, Ls)]))

-La*cos(theta(t))*Derivative(theta(t), t) - Ls*(Derivative(psi(t), t) - Derivative(theta(t), t))*cos(psi(t) - theta(t))
-La*sin(theta(t))*Derivative(theta(t), t) + Ls*(Derivative(psi(t), t) - Derivative(theta(t), t))*sin(psi(t) - theta(t))


In [33]:
print(sm.simplify((v2z.subs([(z, Ls), (Derivative(theta, t), thetad), (Derivative(psi, t), psid)]))))

La**2*thetad(t)**2 + 2*La*Ls*psid(t)*thetad(t)*cos(psi(t)) - 2*La*Ls*thetad(t)**2*cos(psi(t)) + Ls**2*psid(t)**2 - 2*Ls**2*psid(t)*thetad(t) + Ls**2*thetad(t)**2


In [40]:
vpx = vxz.subs(z, Ls)
vpy = vyz.subs(z, Ls)
apx = sm.diff(vpx, t)
apy = sm.diff(vpy, t)
print("apx = ", apx)
print("apy = ", apy)

apx =  La*sin(theta(t))*Derivative(theta(t), t)**2 - La*cos(theta(t))*Derivative(theta(t), (t, 2)) + Ls*(Derivative(psi(t), t) - Derivative(theta(t), t))**2*sin(psi(t) - theta(t)) - Ls*(Derivative(psi(t), (t, 2)) - Derivative(theta(t), (t, 2)))*cos(psi(t) - theta(t))
apy =  -La*sin(theta(t))*Derivative(theta(t), (t, 2)) - La*cos(theta(t))*Derivative(theta(t), t)**2 + Ls*(Derivative(psi(t), t) - Derivative(theta(t), t))**2*cos(psi(t) - theta(t)) + Ls*(Derivative(psi(t), (t, 2)) - Derivative(theta(t), (t, 2)))*sin(psi(t) - theta(t))
